In [1]:
import pandas as pd
import ast
from tqdm import tqdm
import spacy
import os
from sklearn.model_selection import train_test_split
tqdm.pandas()

In [2]:
df = pd.read_csv('project-5-at-2022-03-25-08-26-0715cfef.csv')

In [3]:
df.pmcs.nunique()

65

In [3]:
df.head()

,pmcs,patient_description,id,label,annotator,annotation_id,created_at,updated_at,lead_time
0,PMC7748374,"A 36-year-old woman presented with nausea, cou...",213,"[{""start"": 2, ""end"": 4, ""text"": ""36"", ""labels""...",1,206,2022-03-23T08:24:48.857188Z,2022-03-23T08:24:57.759694Z,406854.459
1,PMC7337161,The patient was a 75-year-old woman with a his...,193,"[{""start"": 18, ""end"": 20, ""text"": ""75"", ""label...",1,186,2022-03-22T16:56:33.265823Z,2022-03-22T16:58:07.616745Z,351244.332
2,PMC7521458,A 31-year-old woman with ALD was admitted to o...,168,"[{""start"": 2, ""end"": 4, ""text"": ""31"", ""labels""...",1,161,2022-03-17T13:29:21.066643Z,2022-03-18T15:21:17.945825Z,701.958
3,PMC7782106,A 34-year-old male with a known history of alc...,166,"[{""start"": 14, ""end"": 18, ""text"": ""male"", ""lab...",1,159,2022-03-14T15:56:53.178265Z,2022-03-14T21:15:11.220487Z,2414.577
4,PMC7782106,A 32-year-old male with a past medical history...,165,"[{""start"": 14, ""end"": 18, ""text"": ""male"", ""lab...",1,158,2022-03-14T15:09:24.876761Z,2022-03-14T21:12:06.952800Z,2230.298


In [4]:
tmp = df.label.apply(ast.literal_eval)
tmp = tmp.to_list()

In [5]:
def create_tuples(markup):
    
    value = ast.literal_eval(markup)
    
    entity_list = []

    for element in value:

        tup = (element['start'], element['end'], element['labels'][0])
        entity_list.append(tup)
            
    return entity_list

In [6]:
df['markup'] = df.label.apply(create_tuples)

In [7]:
df_short = df[['pmcs', 'patient_description', 'markup']]

In [8]:
df_short

,pmcs,patient_description,markup
0,PMC7748374,"A 36-year-old woman presented with nausea, cou...","[(2, 4, Age), (14, 19, Sex), (35, 41, Clinical..."
1,PMC7337161,The patient was a 75-year-old woman with a his...,"[(18, 20, Age), (30, 35, Sex), (54, 62, Clinic..."
2,PMC7521458,A 31-year-old woman with ALD was admitted to o...,"[(2, 4, Age), (14, 19, Sex), (25, 28, Clinical..."
3,PMC7782106,A 34-year-old male with a known history of alc...,"[(14, 18, Sex), (43, 62, Clinical characterist..."
4,PMC7782106,A 32-year-old male with a past medical history...,"[(14, 18, Sex), (50, 69, Clinical characterist..."
...,...,...,...
241,PMC7365853,A 74-year-old woman with a history of remote b...,"[(2, 4, Age), (14, 19, Sex), (38, 179, Clinica..."
242,PMC7126798,"In the fourth case, the patient was a 73 year-...","[(38, 40, Age), (50, 56, Sex), (75, 89, Clinic..."
243,PMC7126798,"In the third case, a 29 year-old housekeeper f...","[(21, 23, Age), (45, 51, Sex), (70, 96, Clinic..."
244,PMC7126798,"In the second case, a 61 year-old female with ...","[(22, 24, Age), (34, 40, Sex), (46, 63, Clinic..."


In [44]:
len(df_short)

246

In [16]:
m = df_short.markup.iloc[0]

In [36]:
pairs = []
for m in df_short.markup.to_list():
    m = sorted(m, key=lambda x: x[0])
    i_0 = m[0]
    for i in m[1:]:
        if i_0[1] >= i[0] and i_0[2] != i[2]:
            pairs.append((i_0[2], i[2]))
        i_0 = i

In [37]:
cnt = 0
for m in df_short.markup.to_list():

    for i in m:
        cnt +=1 


In [38]:
len(pairs), cnt

(80, 6477)

In [40]:
p = list(set(pairs))

In [41]:
len(p)

46

In [43]:
from collections import Counter

Counter(pairs).most_common()

[(('Clinical characteristics', 'Highest temperature, °C'), 7),
 (('CT', 'Comorbidities'), 6),
 (('Oxygen therapy', 'Mechanical Ventilator Status'), 5),
 (('D-Dimer', 'C-reactive protein, mg/dL'), 3),
 (('Lactate dehydrogenase', 'D-Dimer'), 3),
 (('C-reactive protein, mg/dL', 'Erythrocyte sedimentation rate (ESR)'), 3),
 (('Fibrinogen', 'D-Dimer'), 3),
 (('Comorbidities', 'Laboratory findings'), 3),
 (('Troponin I', 'Creatinine mg/dl'), 2),
 (('Creatinine mg/dl', 'Laboratory findings'), 2),
 (('Ferritin', 'Lactate dehydrogenase'), 2),
 (('Laboratory findings', 'Comorbidities'), 2),
 (('X-Ray', 'Comorbidities'), 2),
 (('Laboratory findings', 'Troponin I'), 2),
 (('Erythrocyte sedimentation rate (ESR)', 'Ferritin'), 2),
 (('Mechanical Ventilator Status', 'Oxygen therapy'), 2),
 (('X-Ray', 'CT'), 2),
 (('Ferritin', 'D-Dimer'), 1),
 (('C-reactive protein, mg/dL', 'Lactate dehydrogenase'), 1),
 (('C-reactive protein, mg/dL', 'Ferritin'), 1),
 (('D-Dimer', 'Ferritin'), 1),
 (('Ferritin', 'C-r

In [12]:
df_short.to_excel('thesis_dataframe.xlsx', engine='openpyxl', index=False)

In [13]:
def get_relevant_entity(text, labels):
    res = {}
#     labels = literal_eval(labels)
    if labels:
        for ent in labels:        
            entity = text[ent[0]:ent[1]]
            entity = entity.strip()

            if ent[2] not in res:
                res[ent[2]] = [entity]
            else:
                res[ent[2]].append(entity)
    return res

In [11]:
df_short['ners_true'] = df_short.progress_apply(lambda x: get_relevant_entity(
    text=x['patient_description'], 
    labels=x['markup']
), axis=1)

100%|██████████| 246/246 [00:00<00:00, 33101.24it/s]
/home/user/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [12]:
train, valid = train_test_split(df_short, test_size=0.3, random_state=42)

In [53]:
path = "./data_thesis_for_spacy_txc"
os.makedirs(path, exist_ok=True)

In [46]:
training_data = []

for index, row in tqdm(train.iterrows(), total=len(train)):
    line = row['patient_description']
    ners = row['markup']#literal_eval(row['glued_markup'])
    training_data.append((line, ners))

100%|██████████| 172/172 [00:00<00:00, 24787.67it/s]


In [47]:
valid_data = []

for index, row in tqdm(valid.iterrows(), total=len(valid)):
    line = row['patient_description']
    ners = row['markup']#literal_eval(row['glued_markup'])
    valid_data.append((line, ners))

100%|██████████| 74/74 [00:00<00:00, 23368.36it/s]


In [64]:
from spacy.tokens import DocBin

nlp = spacy.blank("ru")

# the DocBin will store the example documents
db = DocBin()
for text, annotations in tqdm(training_data):
     
    if type(text) == str:
        doc = nlp(text)
        ents = []
        if len(annotations):
            for start, end, label in annotations:
                t = text[start:end]
                if t:
                    span = doc.char_span(start, end, label=label)
                    if span:
                        ents.append(span)
#         doc.ents = ents
# span_lst is now a list of spaCy `Span` objects
        # Set the Spans as document entities
#         doc.set_ents(ents)
        # Set the document entities as spans
        doc.spans['txs'] = ents
        db.add(doc)
    else:
        print(text, annotations)
        
#     break
db.to_disk(f"{path}/train.spacy")

100%|██████████| 172/172 [00:00<00:00, 465.41it/s]


In [66]:
db = DocBin()
for text, annotations in valid_data:
    if type(text) == str:
        doc = nlp(text)
        ents = []
        if len(annotations):
            for start, end, label in annotations:
                t = text[start:end]
                if t:
                    span = doc.char_span(start, end, label=label)
                    if span:
                        ents.append(span)
#         doc.set_ents(ents)
        # Set the document entities as spans
        doc.spans['txs'] = ents

        db.add(doc)
    else:
        print(text, annotations)
db.to_disk(f"{path}/valid.spacy")

In [24]:
true_labels = set()
for markup in valid['markup'].to_list():
    for m in markup:
        true_labels.add(m[2])

In [26]:
def get_label_set(nlp):
    labels = set()

    for text in tqdm(valid["patient_description"].tolist()):

        doc = nlp(text)

        e = doc.spans['txs']


        for i in e:

            labels.add(i.label_)
    
    print(f'Found {len(labels)} out of {len(true_labels)}')
    return labels


In [28]:
nlp = spacy.load("data_thesis_for_spacy_txc/cpu/model-best")
get_label_set(nlp)

100%|██████████| 74/74 [00:08<00:00,  8.44it/s]

Found 9 out of 43


{'COVID-19 medication',
 'Clinical characteristics',
 'Comorbidities',
 'Death',
 'Highest temperature, °C',
 'Laboratory findings',
 'Oxygen saturation',
 'Sex',
 'Therapy'}

In [30]:
nlp = spacy.load("data_thesis_for_spacy_txc/bert/model-best")
get_label_set(nlp)

100%|██████████| 74/74 [00:54<00:00,  1.37it/s]

Found 9 out of 43


{'COVID-19 medication',
 'Clinical characteristics',
 'Comorbidities',
 'Death',
 'Highest temperature, °C',
 'Laboratory findings',
 'Oxygen saturation',
 'Sex',
 'Therapy'}

In [32]:
nlp = spacy.load("data_thesis_for_spacy_txc/clinicalbert/model-best")
get_label_set(nlp)

100%|██████████| 74/74 [00:56<00:00,  1.31it/s]

Found 10 out of 43


{'COVID-19 medication',
 'Clinical characteristics',
 'Comorbidities',
 'Death',
 'Highest temperature, °C',
 'Laboratory findings',
 'Mechanical Ventilator Status',
 'Oxygen saturation',
 'Sex',
 'Therapy'}

In [33]:
nlp = spacy.load("data_thesis_for_spacy_txc/BiomedNLP_fulltext/model-best")
get_label_set(nlp)

100%|██████████| 74/74 [00:45<00:00,  1.62it/s]

Found 6 out of 43


{'COVID-19 medication',
 'Clinical characteristics',
 'Comorbidities',
 'Death',
 'Sex',
 'Therapy'}

In [34]:
nlp = spacy.load("data_thesis_for_spacy_txc/Bio_Discharge_Summary_BERT/model-best")
get_label_set(nlp)

100%|██████████| 74/74 [00:51<00:00,  1.44it/s]

Found 9 out of 43


{'COVID-19 medication',
 'Clinical characteristics',
 'Comorbidities',
 'Death',
 'Highest temperature, °C',
 'Laboratory findings',
 'Oxygen saturation',
 'Sex',
 'Therapy'}

In [35]:
nlp = spacy.load("data_thesis_for_spacy_txc/bioclinicalBERT-finetuned-covid-papers/model-best")
get_label_set(nlp)

100%|██████████| 74/74 [00:50<00:00,  1.45it/s]

Found 12 out of 43


{'COVID-19 medication',
 'Clinical characteristics',
 'Comorbidities',
 'Death',
 'Highest temperature, °C',
 'Hospital LOS (d)',
 'Laboratory findings',
 'Mechanical Ventilator Status',
 'Oxygen saturation',
 'Oxygen therapy',
 'Sex',
 'Therapy'}

In [36]:
nlp = spacy.load("data_thesis_for_spacy_txc/BiomedNLP-PubMedBERT-base-uncased-abstract/model-best")
get_label_set(nlp)

100%|██████████| 74/74 [00:44<00:00,  1.65it/s]

Found 8 out of 43


{'COVID-19 medication',
 'Clinical characteristics',
 'Comorbidities',
 'Death',
 'Highest temperature, °C',
 'Laboratory findings',
 'Sex',
 'Therapy'}

In [ ]:
nlp = spacy.load("data_thesis_for_spacy_txc/clinical-pubmed-bert-base-512/model-best")
get_label_set(nlp)

 92%|█████████▏| 68/74 [00:40<00:03,  1.62it/s]